In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip uninstall clr
!pip install pythonnet
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('corrected_labelled.csv')
df = df.dropna()

df['nps'] = df['nps'].replace('10 (Extremely likely)',10)
df['nps'] = df['nps'].replace('0 (Not at all likely)',0)
df['nps'] = df['nps'].astype(int)

#target variable will nps split into demoters, passives and promoters
df['label'] = np.where(df['nps'] == 3,2,
                  np.where(df['nps'] == 1,0,1))

gb = df.groupby('label')['label'].count()
print(gb)

df.info()

label
0    411
1    159
2    565
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   response       1135 non-null   object
 1   nps            1135 non-null   int64 
 2   Term           1135 non-null   object
 3   ticket_number  1135 non-null   object
 4   grp            1135 non-null   object
 5   label          1135 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 53.3+ KB


In [ ]:
import re
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()

df['response'] = df['response'].apply(lambda x: text_preprocessing(x))
df['Term'] = df['Term'].apply(lambda x: text_preprocessing(x))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df[['response','Term']],df['label'], test_size = 0.33, random_state=42)

#X_test, X_val, y_test, y_val = train_test_split(X_test,y_test, test_size = 0.5, random_state=42)

In [ ]:
#calculating class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight = 'balanced', classes = [2,1,0], y = y_train)
type(class_weights)

numpy.ndarray

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

train = X_train
train['label'] = y_train.to_list()

# test = X_test
# test['label'] = y_test.to_list()

val = X_val
val['label'] = y_val.to_list()

train_ds = Dataset.from_pandas(train)
#test_ds = Dataset.from_pandas(test)
val_ds = Dataset.from_pandas(val)

ds = DatasetDict()

ds['train'] = train_ds
#ds['test'] = test_ds
ds['validation'] = val_ds
ds

DatasetDict({
    train: Dataset({
        features: ['response', 'Term', 'label', '__index_level_0__'],
        num_rows: 760
    })
    validation: Dataset({
        features: ['response', 'Term', 'label', '__index_level_0__'],
        num_rows: 375
    })
})

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = False)


def tokenize_function(example):
    return tokenizer(example["response"], example["Term"], truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["response", "Term", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 46]),
 'token_type_ids': torch.Size([8, 46]),
 'attention_mask': torch.Size([8, 46])}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.2359, grad_fn=<NllLossBackward0>) torch.Size([8, 3])


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

285


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:

from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_predicted_labels = []
    all_true_labels = []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            b_labels = batch['labels']
            b_input_ids = batch['input_ids']
            b_attn_mask = batch['attention_mask']

            logits = model(**batch)
            loss = loss_fn(logits['logits'], b_labels)

            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_labels = torch.max(logits['logits'], 1)
            correct_predictions += (predicted_labels == b_labels).sum().item()
            total_samples += b_labels.size(0)

            all_predicted_labels.extend(predicted_labels.cpu().numpy())
            all_true_labels.extend(b_labels.cpu().numpy())

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples

    # Calculate weighted F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(b_labels.cpu().numpy(), predicted_labels.cpu().numpy(), average='weighted')

    return average_loss, accuracy, precision, recall, f1


In [ ]:
from tqdm.auto import tqdm
import torch.nn as nn

progress_bar = tqdm(range(num_training_steps))

tensor_weights = torch.tensor(np.float32(class_weights)).to(device)
loss_fn = nn.CrossEntropyLoss(weight = tensor_weights)
model.train()

epoch_list = []
step_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_recall_list = []
val_prec_list = []
val_f1_list = []



evaluate_every_n_steps = 20
for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        b_labels = batch['labels']
        b_input_ids = batch['input_ids']
        b_attn_mask = batch['attention_mask']
        #logits = model(b_input_ids, b_attn_mask,)
        logits = model(**batch)
        loss = loss_fn(logits['logits'],b_labels)
        #print(f'training loss: {loss}')

        #validation loss
        if ((step) % evaluate_every_n_steps == 0) or (step == len(train_dataloader) - 1):
          val_loss, val_accuracy, val_prec, val_recall, val_f1 = evaluate_model(model, eval_dataloader, loss_fn, device)
          print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
          # Optionally, you can store the loss and accuracy values for later plotting.
          # Add code for plotting as needed.
          epoch_list.append(epoch + 1)
          step_list.append(step)
          train_loss_list.append(loss.item())
          val_loss_list.append(val_loss)
          val_acc_list.append(val_accuracy)
          val_recall_list.append(val_recall)
          val_prec_list.append(val_prec)
          val_f1_list.append(val_f1)


        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

loss_info = pd.DataFrame(list(zip(epoch_list, step_list, train_loss_list, val_loss_list, val_acc_list, val_recall_list, val_prec_list, val_f1_list)),
                         columns = ['epoch', 'step', 'train loss', 'val loss', 'val accuracy','val recall', 'val precision', 'val f1'])

  0%|          | 0/285 [00:00<?, ?it/s]

Epoch 1/3, Validation Loss: 1.0467167766071221, Validation Accuracy: 0.744
Epoch 1/3, Validation Loss: 0.6068052062962918, Validation Accuracy: 0.8933333333333333
Epoch 1/3, Validation Loss: 0.42822579754159806, Validation Accuracy: 0.8773333333333333
Epoch 1/3, Validation Loss: 0.4102235285050057, Validation Accuracy: 0.896
Epoch 1/3, Validation Loss: 0.3658793878919901, Validation Accuracy: 0.8533333333333334
Epoch 1/3, Validation Loss: 0.4078628439059917, Validation Accuracy: 0.8693333333333333
Epoch 2/3, Validation Loss: 0.39392771777954505, Validation Accuracy: 0.8613333333333333
Epoch 2/3, Validation Loss: 0.3806370258965391, Validation Accuracy: 0.888
Epoch 2/3, Validation Loss: 0.3938190253808143, Validation Accuracy: 0.8933333333333333
Epoch 2/3, Validation Loss: 0.3867417628619265, Validation Accuracy: 0.8933333333333333
Epoch 2/3, Validation Loss: 0.3971034823421468, Validation Accuracy: 0.888
Epoch 2/3, Validation Loss: 0.3974674897704353, Validation Accuracy: 0.888
Epoch 3

In [ ]:
loss_info

,epoch,step,train loss,val loss,val accuracy,val recall,val precision,val f1
0,1,0,1.456655,1.046717,0.744000,1.000000,1.000000,1.000000
1,1,20,0.614615,0.606805,0.893333,1.000000,1.000000,1.000000
2,1,40,0.814892,0.428226,0.877333,0.857143,0.928571,0.866667
3,1,60,0.752945,0.410224,0.896000,1.000000,1.000000,1.000000
4,1,80,1.016846,0.365879,0.853333,1.000000,1.000000,1.000000
5,1,94,1.137646,0.407863,0.869333,1.000000,1.000000,1.000000
6,2,0,0.223271,0.393928,0.861333,1.000000,1.000000,1.000000
7,2,20,0.651610,0.380637,0.888000,1.000000,1.000000,1.000000
8,2,40,0.035958,0.393819,0.893333,1.000000,1.000000,1.000000
9,2,60,0.056454,0.386742,0.893333,1.000000,1.000000,1.000000


In [ ]:
len(eval_dataloader)


11

In [ ]:
import evaluate

#metric = evaluate.load("glue", "mrpc", average = "weighted")
# metric = evaluate.combine([
#     evaluate.load("precision", average="macro"),
#     evaluate.load("recall", average="macro")
# ])

metric = evaluate.load("accuracy")
model.eval()

all_logits = []
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    all_logits.append(logits)

    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())


all_logits = torch.cat(all_logits, dim=0)
probs = torch.argmax(all_logits, dim=1).cpu().numpy()

{'accuracy': 0.8986666666666666}


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_test = val['label'].to_numpy()

y_pred = val['predicted_class'].to_numpy()

class_labels = ['Negative','Neutral','Positive']

# Create the confusion matrix with custom labels
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[119  11   0]
 [ 12  24   7]
 [  1   7 194]]


In [ ]:
# Display the classification report with custom labels
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


Classification Report:
              precision    recall  f1-score   support

    Negative       0.90      0.92      0.91       130
     Neutral       0.57      0.56      0.56        43
    Positive       0.97      0.96      0.96       202

    accuracy                           0.90       375
   macro avg       0.81      0.81      0.81       375
weighted avg       0.90      0.90      0.90       375



In [ ]:
probs
val['predicted_class'] = probs

#val.to_csv('check.csv')

In [ ]:
val

,response,Term,label,predicted_class
784,coach was delayed by an hour causing me to mis...,delayed,0,0
900,journey ringwood to london the coach was far t...,journey,1,0
413,punctuality,punctuality,2,2
467,since the seats are too tight and extremely fi...,seats,0,0
746,the 2nd stage of the journey to folkestone was...,late,0,0
...,...,...,...,...
33,i was pleased with the comfort of the bus and ...,phone,1,1
754,it was a good experience despite the late wait...,delay,0,0
575,always late,late,0,0
259,"friendly staff, efficient, reasonable price",price,2,2


### uploading to HF checkpoint

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
tokenizer.push_to_hub("absa-transport")

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nxnag/absa-transport/commit/6275f9f5fecb065282d0cd456ba1a14b49d7c896', commit_message='Upload tokenizer', commit_description='', oid='6275f9f5fecb065282d0cd456ba1a14b49d7c896', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("absa-transport")

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nxnag/absa-transport/commit/65cb131a31f5d72252898600a69ecb9fa63383e6', commit_message='Upload DebertaV2ForSequenceClassification', commit_description='', oid='65cb131a31f5d72252898600a69ecb9fa63383e6', pr_url=None, pr_revision=None, pr_num=None)

**Saving checkpoint to drive**

In [ ]:
#### Saving ###
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /content/gdrive/My\ Drive/FOLDER

model_save_name = 'asba_classifier.pt'
path = F"/content/gdrive/MyDrive/{model_save_name}"
#path = F"/Shared drives/CRM & Insight/Analysis/arun/Text_Analytics"
torch.save(model.state_dict(), path)

Mounted at /content/gdrive


**Predicting term and response absa sentiment**

Going from pandas df with responses and terms, converting data types to get model outputs and then putting it back into the dataframe.

In [ ]:
#preprocessing to pass into bert
df = pd.read_csv('emotion_append.csv')
df = df.dropna(subset = ['response','Term'])

df['nps'] = df['nps'].replace('10 (Extremely likely)',10)
df['nps'] = df['nps'].replace('0 (Not at all likely)',0)
df['nps'] = df['nps'].astype(int)

#target variable will nps split into demoters, passives and promoters
df['label'] = np.where(df['nps'] >= 9,2,
                  np.where(df['nps'] <= 6,0,1))

df['response'] = df['response'].apply(lambda x: text_preprocessing(x))
df['Term'] = df['Term'].apply(lambda x: text_preprocessing(x))

In [ ]:
prob_df = df[['response','Term', 'label']]

ds = Dataset.from_pandas(prob_df)

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["response"], example["Term"], truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3359 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
#detokenizing inputs
# token_output = []
# for tok in tokenized_datasets['input_ids']:
#   tokens = []
#   detokenized = tokenizer.decode(tok)
#   token_output.append(detokenized)

In [ ]:
#tokenized_datasets = tokenized_datasets.remove_columns(["response", "Term"])
tokenized_datasets = tokenized_datasets.remove_columns(["response", "Term", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    tokenized_datasets, shuffle=True, batch_size=8, collate_fn=data_collator
)
for batch in  dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 52]),
 'token_type_ids': torch.Size([8, 52]),
 'attention_mask': torch.Size([8, 52])}

In [ ]:
# import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# device

In [ ]:
all_logits = []
input_ids = []
for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    input_ids.append(batch['input_ids'])
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    all_logits.append(logits)

all_logits = torch.cat(all_logits, dim=0)
probs = torch.argmax(all_logits, dim=1).cpu().numpy()
probs

array([2, 0, 0, ..., 0, 2, 1])

In [ ]:
token_output = []
for tok in tokenized_datasets['input_ids']:
  tokens = []
  detokenized = tokenizer.decode(tok)
  token_output.append(detokenized)

In [ ]:
token_output = []
for batch in input_ids:
  for id in batch:
      token_output.append(tokenizer.decode(id))

In [ ]:
token_output
check = pd.DataFrame(list(zip(token_output)), columns = ['detokenized'])
check.to_csv('check.csv')

In [ ]:
df['aspect_sentiment'] = probs
df.to_csv('asba_append.csv')

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,ticket_number,response,nps,csat,Term,Tier_2,Tier_1,entities,entity_type,entity_start,entity_end,num,emotion,label,aspect_sentiment
0,0,0,CSA03746,i think the driver was kevin. lgw to lhr. an a...,10,Very satisfied,driver,Staff,People,"{'entity': 'B-PER', 'score': 0.99710387, 'inde...",B-PER,23,28,1,neutral,2,2
1,1,16195,EUAJWU72,the trip from portsmouth to london was affecte...,7,Somewhat dissatisfied,delayed,Time,Service,"{'entity': 'B-PER', 'score': 0.9990357, 'index...",B-PER,125,132,1,joy,1,0
2,2,7926,EUADWE24,left on time and the driver (tom) was very pol...,10,Very satisfied,time,Time,Service,"{'entity': 'B-PER', 'score': 0.9992095, 'index...",B-PER,29,32,1,neutral,2,0
3,3,13909,EUAHLT03,exceptionally friendly & humorous drivers (not...,10,Very satisfied,journey,Journey,Service,"{'entity': 'B-PER', 'score': 0.9753795, 'index...",B-PER,79,81,1,joy,2,1
4,4,16196,EUAJWU72,the trip from portsmouth to london was affecte...,7,Somewhat dissatisfied,driver,Staff,People,"{'entity': 'B-PER', 'score': 0.9990357, 'index...",B-PER,125,132,1,joy,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639,3639,6424,EUADAR88,"arrived at liverpool,the coach was delayed so ...",0,Very dissatisfied,arrived,Journey,Service,"{'entity': 'B-LOC', 'score': 0.8416962, 'index...",B-LOC,11,20,2,sadness,0,0
3640,3640,6423,EUADAR88,"arrived at liverpool,the coach was delayed so ...",0,Very dissatisfied,delayed,Time,Service,"{'entity': 'B-LOC', 'score': 0.8416962, 'index...",B-LOC,11,20,2,sadness,0,2
3641,3641,6422,EUADAR74,coach on time at heathrow and arrived on time ...,8,Very satisfied,arrived,Journey,Service,"{'entity': 'I-LOC', 'score': 0.9968671, 'index...",I-LOC,109,111,2,sadness,1,0
3642,3642,6481,EUADBM85,to get to london it took us 6 hours. then comi...,1,Very dissatisfied,hour,Time,Service,"{'entity': 'B-LOC', 'score': 0.99972266, 'inde...",B-LOC,10,16,2,surprise,0,2
